# Deploy Web application using "Azure App services using Docker image"

In this exercise we will learn how to deploy our web app by creating a Docker image and deploy the docker image using "Azure App services". As a prerequisite we assume you have created the [Azure cloud account](https://portal.azure.com). Along with that, we also assume you have installed Docker in your host development system. [Here is a quick link on how to get started on docker](https://docs.docker.com/get-started/).

We will also learn 
* How to create a docker container registry in Azure environment.
* How to create a docker image with our web application and push it in to docker container registry.
* How to publish the docker image as web service in Azure environment.

## Create a Docker Container Registry

Docker container registry in a high level is something similar to github. In github we store the source code files of our project. Where as we store Docker images in the container registry. Docker in itself is a vast subject to cover. We just breeze on few basic concepts to get you started, but we highly recommend you to go through the [official documents](https://docs.docker.com/develop/) to get better understanding of how docker system works and how you can use it in developing your applications.

As usual we start from our dash board. We search for "Container registries" in search bar. Select the option as show in the image below.
<br>

![](images/docker/step_1.png)

![](images/docker/step_2.png)

![](images/docker/step_3.png)

![](images/docker/step_4.png)


![](images/docker/step_5.png)

Give some meaning full name to your container.
<br>

![](images/docker/step_6.png)

![](images/docker/step_7.png)

![](images/docker/step_8.png)

It is important to note that, validation should pass. If there are any errors, you will not be able to create the required "Container registry".
<br>

![](images/docker/step_9.png)

![](images/docker/step_10.png)

### Get Container access keys

Our Container is ready! Now we need to upload our docker image. (We have not created a docker image yet. We will create one shortly). But before we upload, we need to enable the admin user and get the "username" and "password" details to push our docker image in to the newly created "Container registry".
<br>

![](images/docker/step_11.png)

By default, "Admin" user is disabled. We need to enable it manually to get our user name and password. Once you enable the "Admin" user your "username and password" will be displayed as shown below in the image. Please make a note of your "username" and "password", we will be using them in next few steps.
<br>

![](images/docker/step_12.png)

## Create a docker image

We assume you have installed required docker dependencies in your system. If not, [Here is a quick link on how to get started on docker](https://docs.docker.com/get-started/)

Let us clone and change directory to our repository.

```sh
$ git clone https://github.com/bigvisionai/pytorch-web-app-deploy-azure.git
$ cd pytorch-web-app-deploy-azure
```
Here are the contents of our project directory.
```sh
$ ls
config.py  Dockerfile  flask_pytorch_web_app  local_run.sh docker_run.sh  README.md  requirements.txt  setup.py  torchapp.ini  wsgi.py
```
To generate a Docker image we need to have a file called `Dockerfile` in our project. Let us have a closer look at the contents of the `Dockerfile`
<br>

```sh
$ cat Dockerfile 
FROM python:3.8.2-slim

COPY requirements.txt /
RUN pip3 install -r /requirements.txt

COPY . /app
WORKDIR /app
EXPOSE 80
CMD [ "bash", "docker_run.sh" ]

```

Let us dissect the `Dockerfile`.

We are using the official `python` [docker image](https://hub.docker.com/_/python) as our base image. Specifically version `3.8.2-slim`

We copy our requirements.txt file in to our docker image `COPY requirements.txt /` to install the dependencies of our web application.

We inform the docker to do the installation of dependencies using `RUN pip3 install -r /requirements.txt`.

Next, we will inform the docker to copy all the files of our application's current directory `.` in to the path `/app`.

We make `/app` as the working directory, `WORKDIR /app`. 

We need to expose the port on which our application will be listening. `EXOPSE 80` exposes the `port 80` for our application.

Now we run our script as we did during our application development process. `CMD [ "bash", "docker_run.sh" ]`. Since `docker_run.sh` is a shell script, we are using the `bash` command to execute the shell script.

### Build and test the docker image locally

Now that we understand how the `Dockerfile` is organized. We need to initiate the `docker build` to generate our docker image. 
Command to build a docker image goes as follows.

```sh
$ sudo docker build --tag <name_of_your_docker_image> <path_to_Dockerfile> 
```
**For Example:**

```sh
$ sudo docker build --tag imagenetpytorchwebapp .
```
Above command will initiate the build and we observe the logs as shown below.
```sh
Sending build context to Docker daemon  351.7kB
Step 1/7 : FROM python:3.8.2-slim
 ---> e7d894e42148
Step 2/7 : COPY requirements.txt /
 ---> Using cache
 ---> 6280f67bd009
Step 3/7 : RUN pip3 install -r /requirements.txt
 ---> Using cache
 ---> fd887298df22
Step 4/7 : COPY . /app
 ---> ad9727c457b1
Step 5/7 : WORKDIR /app
 ---> Running in 9ebb75c6a812
Removing intermediate container 9ebb75c6a812
 ---> 1b9d3968cf2d
Step 6/7 : EXPOSE 80
 ---> Running in 4363b7319564
Removing intermediate container 4363b7319564
 ---> 1368d4cc1bde
Step 7/7 : CMD [ "bash", "docker_run.sh" ]
 ---> Running in f6fe61069090
Removing intermediate container f6fe61069090
 ---> d7cef53083f4
Successfully built d7cef53083f4
Successfully tagged imagenetpytorchwebapp:latest
```
Once our docker image is built we can run our image and test if our application is running through the docker image.

**NOTE:** If you have done the virtual machine deployment approach on the same machine. Probably your nginx service would be still running and would be occupying port 80. Make sure you stop the nginx serivce (`sudo service nginx stop`).

We run the following command to test our docker.

```sh
$ sudo docker run --publish 80:80 --detach --name localtesting imagenetpytorchwebapp 
```
Here is the breakup of the command.

`sudo docker run` to run the docker.
`--publish 80:80` to tell the docker to publish the request arriving at Host machine's 80 to docker's port 80.
`--detach` to inform the docker to detach it and run it as a separate process.
`--name localtesting` Because we are detaching our app, we want it easier for us to navigate and identify the our docker image hence we use `--name` option and give a name of our choice. In our case, we call it `localtesting`. 
Finally we tell which image to run. In our case we want `imagenetpytorchwebapp` to be run.

At any point to get the list of docker image running in your system you can run the following command.

```sh
$ sudo docker ps
CONTAINER ID        IMAGE                   COMMAND               CREATED             STATUS              PORTS                NAMES
e78d874b1e6a        imagenetpytorchwebapp   "bash docker_run.sh"   13 seconds ago      Up 10 seconds       0.0.0.0:80->80/tcp   localtesting

```

You can go to browser and type `localhost` and our application should be visible and you should be able to test it as earlier.

## Push the docker image in to "Container registries"
Now, that we have created an image, we would like to push it to the docker container registries which we created in our previous step. Go back to your Azure portal. Navigate to your "Container registry" resource which you created. Click on "Quick Start" button on the left hand side panel. Note down the command as highlighted.
<br>

![](images/docker/step_14.png)

<br>
In our example, the container registry is called `opencvcontainer.azurecr.io` you need to copy name of your container registry as shown in the above image.

Now run the command 

```sh
sudo docker login <your_container_registry_uri>
Username: <your_container_registry_user_name>
Password: <your_container_registry_password>
```
If the login is successful you will see the log as shown below.
```sh
Login Succeeded
```
**NOTE:** You can get the `username` and `password` from the previous step **"Get Container access keys"**. 

**Example:**
```sh
sudo docker login opencvcontainer.azurecr.io
Username: opencvcontainer
Password: 
Login Succeeded
```

Once we have logged in, we need to tag our container and push it to the container registry.
```sh
$ sudo docker tag <your_docker_image_name> <your_container_registry_uri>/<your_docker_image_name>

```

**Example:**

```sh
$ sudo docker tag imagenetpytorchwebapp opencvcontainer.azurecr.io/imagenetpytorchwebapp
```

<br>

![](images/docker/step_15.png)

Finally we can use the `docker push` command to push the docker image to our container registry.
```sh
sudo docker push  <your_container_registry_uri>/<your_docker_image_name>

```
**Example:**

```sh
$ sudo docker push opencvcontainer.azurecr.io/imagenetpytorchwebapp
```
<br>
The docker image will be uploaded in to your registry. It will take some time to push the image in to the container registry. You should start observing the logs something similar to the one shown below.
<br>

```sh
The push refers to repository [opencvcontainer.azurecr.io/imagenetpytorchwebapp]
dc5de5bf80a9: Pushed 
2cdddcf57cce: Pushed 
fdc7e4b93ce0: Pushed 
a9e858adafbc: Pushed 
62cdbc1bc81e: Pushed 
dac74b28432f: Pushed 
6376837eded8: Pushed 
c2adabaecedb: Pushed 

latest: digest: sha256:a794d68d9ec4364ff3ca63d576432394742cb961f7c32125072d5a071b32dfc4 size: 2001
```

Once the `docker push` is successful. We can verify the same by again navigating back to our container registry resource in the Azure portal.
<br>

![](images/docker/step_16.png)

## Create an App service using Docker image
<br>
Similar to our Application deployment using "App service" method, we need to create the resources, this time we use the docker image which we just pushed in to our container registry.
<br>

![](images/docker/step_17.png)

![](images/docker/step_18.png)

![](images/docker/step_19.png)

![](images/docker/step_20.png)

![](images/docker/step_21.png)

We select the "Docker Container" as our option for "Publish" field.
<br>

![](images/docker/step_22.png)

![](images/docker/step_23.png)

![](images/docker/step_24.png)

<br>
It is time to link the docker image.

![](images/docker/step_25_a.png)

Select the source of the image container as "Azure Container Registry"
<br>

![](images/docker/step_25_b.png)

Since we have only one docker image in our container registery there is nothing much to do. But if you already have multiple images and different versions of the same app, then care should be taken to select the appropriate options.
<br>

![](images/docker/step_25.png)

![](images/docker/step_27.png)

It will take some time to get the app linked.  Once it is successful you can go back to the overview page. and navigate to our app link as shown below.
<br>

![](images/docker/step_28.png)

![](images/docker/step_29.png)

![](images/docker/step_30.png)

## Short cut approach to load the app using docker image.

There is also a shortcut approach we can use to quickly deploy the app. 

![](images/docker/shortcut_1.png)
![](images/docker/shortcut_2.png)